Загружаю датасет

In [1]:

from config import *
import json
from sys import getsizeof
import pandas as pd
import datetime

In [2]:
df = pd.read_csv(DATASETS_ABS_PATHS["BWSAS"])
df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True) # Replace spaces with underscores in column names
df.Measurement_Timestamp = pd.to_datetime(df.Measurement_Timestamp, format="%m/%d/%Y %I:%M:%S %p")
df = df.convert_dtypes()
df.dtypes

Station_Name                   string[python]
Measurement_Timestamp          datetime64[ns]
Air_Temperature                       Float64
Wet_Bulb_Temperature                  Float64
Humidity                                Int64
Rain_Intensity                        Float64
Interval_Rain                         Float64
Total_Rain                            Float64
Precipitation_Type                      Int64
Wind_Direction                          Int64
Wind_Speed                            Float64
Maximum_Wind_Speed                    Float64
Barometric_Pressure                   Float64
Solar_Radiation                         Int64
Heading                                 Int64
Battery_Life                          Float64
Measurement_Timestamp_Label    string[python]
Measurement_ID                 string[python]
dtype: object

In [3]:
df

,Station_Name,Measurement_Timestamp,Air_Temperature,Wet_Bulb_Temperature,Humidity,Rain_Intensity,Interval_Rain,Total_Rain,Precipitation_Type,Wind_Direction,Wind_Speed,Maximum_Wind_Speed,Barometric_Pressure,Solar_Radiation,Heading,Battery_Life,Measurement_Timestamp_Label,Measurement_ID
0,Oak Street Weather Station,2015-05-22 15:00:00,<NA>,7.0,55,0.0,0.0,1.4,0,63,1.9,2.8,<NA>,780,322,12.0,05/22/2015 3:00 PM,OakStreetWeatherStation201505221500
1,Oak Street Weather Station,2015-05-22 17:00:00,<NA>,6.3,56,0.0,0.0,1.4,0,124,1.5,2.3,<NA>,180,322,12.1,05/22/2015 5:00 PM,OakStreetWeatherStation201505221700
2,Oak Street Weather Station,2015-05-22 18:00:00,<NA>,6.5,54,0.0,0.0,1.4,0,156,1.9,3.4,<NA>,127,322,12.1,05/22/2015 6:00 PM,OakStreetWeatherStation201505221800
3,Oak Street Weather Station,2015-05-22 19:00:00,<NA>,6.3,53,0.0,0.0,1.4,0,150,1.4,4.5,<NA>,67,322,12.1,05/22/2015 7:00 PM,OakStreetWeatherStation201505221900
4,Oak Street Weather Station,2015-05-22 20:00:00,<NA>,6.4,52,0.0,0.0,1.4,0,155,1.1,2.3,<NA>,10,322,12.0,05/22/2015 8:00 PM,OakStreetWeatherStation201505222000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59139,Oak Street Weather Station,2017-11-08 00:00:00,4.7,1.8,60,0.0,0.0,0.0,0,273,0.8,1.5,1005.4,0,2,11.9,11/08/2017 12:00 AM,OakStreetWeatherStation201711082400
59140,Foster Weather Station,2017-11-08 00:00:00,2.89,<NA>,58,<NA>,0.0,<NA>,<NA>,278,2.1,2.3,1004.4,0,<NA>,15.2,11/08/2017 12:00 AM,FosterWeatherStation201711082400
59141,63rd Street Weather Station,2017-11-08 01:00:00,3.4,1.1,66,0.0,0.0,28.8,0,299,1.1,2.2,1004.6,4,354,11.8,11/08/2017 1:00 AM,63rdStreetWeatherStation201711080100
59142,Oak Street Weather Station,2017-11-08 01:00:00,4.2,1.4,59,0.0,0.0,0.0,0,276,0.8,1.7,1005.4,1,2,12.0,11/08/2017 1:00 AM,OakStreetWeatherStation201711080100


Динамически создаю .proto файл для него (не работает)

In [4]:
df.to_dict()

{'Station_Name': {0: 'Oak Street Weather Station',
  1: 'Oak Street Weather Station',
  2: 'Oak Street Weather Station',
  3: 'Oak Street Weather Station',
  4: 'Oak Street Weather Station',
  5: 'Oak Street Weather Station',
  6: 'Oak Street Weather Station',
  7: 'Oak Street Weather Station',
  8: 'Oak Street Weather Station',
  9: 'Oak Street Weather Station',
  10: 'Oak Street Weather Station',
  11: 'Oak Street Weather Station',
  12: 'Oak Street Weather Station',
  13: 'Oak Street Weather Station',
  14: 'Oak Street Weather Station',
  15: 'Oak Street Weather Station',
  16: 'Oak Street Weather Station',
  17: 'Oak Street Weather Station',
  18: 'Oak Street Weather Station',
  19: 'Oak Street Weather Station',
  20: 'Oak Street Weather Station',
  21: 'Oak Street Weather Station',
  22: 'Oak Street Weather Station',
  23: 'Oak Street Weather Station',
  24: 'Oak Street Weather Station',
  25: 'Oak Street Weather Station',
  26: 'Oak Street Weather Station',
  27: 'Oak Street Weat

In [5]:
import pandas as pd
from google.protobuf.descriptor_pb2 import FieldDescriptorProto
from google.protobuf.descriptor_pb2 import FileDescriptorProto
from google.protobuf.compiler.plugin_pb2 import CodeGeneratorRequest
from google.protobuf.compiler.plugin_pb2 import CodeGeneratorResponse

def create_protobuf_template(df):
    # Create a FileDescriptorProto
    file_descriptor_proto = FileDescriptorProto()

    # Create a message type for the DataFrame
    message_type = file_descriptor_proto.message_type.add()
    message_type.name = "DataFrame"

    # Iterate over the columns of the DataFrame
    for column_name, column_type in df.dtypes.items():
        # Create a field descriptor for each column
        field_descriptor = message_type.field.add()
        field_descriptor.name = column_name

        # Map Pandas dtype to Protobuf field type
        if column_type == "int64":
            field_descriptor.type = FieldDescriptorProto.TYPE_INT64
        elif column_type == "float64":
            field_descriptor.type = FieldDescriptorProto.TYPE_DOUBLE
        elif column_type == "bool":
            field_descriptor.type = FieldDescriptorProto.TYPE_BOOL
        elif column_type == "object":
            field_descriptor.type = FieldDescriptorProto.TYPE_STRING
        elif column_type == "datetime64[ns]":
            field_descriptor.type = FieldDescriptorProto.TYPE_STRING
        else:
            field_descriptor.type = FieldDescriptorProto.TYPE_STRING

    # Return the serialized FileDescriptorProto
    return file_descriptor_proto.SerializeToString()

In [6]:
protobuf_template = create_protobuf_template(df)
print(protobuf_template)

b'"\x82\x03\n\tDataFrame\x12\x10\n\x0cStation_Name(\t\x12\x19\n\x15Measurement_Timestamp(\t\x12\x13\n\x0fAir_Temperature(\t\x12\x18\n\x14Wet_Bulb_Temperature(\t\x12\x0c\n\x08Humidity(\t\x12\x12\n\x0eRain_Intensity(\t\x12\x11\n\rInterval_Rain(\t\x12\x0e\n\nTotal_Rain(\t\x12\x16\n\x12Precipitation_Type(\t\x12\x12\n\x0eWind_Direction(\t\x12\x0e\n\nWind_Speed(\t\x12\x16\n\x12Maximum_Wind_Speed(\t\x12\x17\n\x13Barometric_Pressure(\t\x12\x13\n\x0fSolar_Radiation(\t\x12\x0b\n\x07Heading(\t\x12\x10\n\x0cBattery_Life(\t\x12\x1f\n\x1bMeasurement_Timestamp_Label(\t\x12\x12\n\x0eMeasurement_ID(\t'


In [7]:
# Write the byte string to a file.
with open(path.join(PROTOS_FOLDER_ABS_PATH, "protobuf_example.proto"), "wb") as f:
    f.write(protobuf_template)

Компилирую .proto файл

In [8]:
import os

M_BEACH_PROTO = path.join(PROTOS_FOLDER, "m_beach.proto")

compile_proto_command = f"{PROTOC_EXE} --python_out=. {M_BEACH_PROTO}"
os.system(compile_proto_command)

0

Заполняю сообщение данными

In [9]:
def timestamp2unix(timestamp):
    seconds = int(timestamp.timestamp())
    nanos = int(timestamp.timestamp() % 1 * 1e9)

    return (seconds, nanos)

In [10]:
import protos.m_beach_pb2 as m_beach_pb2

m_beach_message = m_beach_pb2.m_beach_message() 

for field, value in df.to_dict(orient='records')[0].items():
    if value is None:
        continue
    if type(value) is pd.Timestamp:
        seconds, nanos = timestamp2unix(value)
        setattr(getattr(m_beach_message, field), "seconds", seconds)
        setattr(getattr(m_beach_message, field), "nanos", nanos)
    else:
        try:
            setattr(m_beach_message, field, value)
        except Exception as e:
            print(f"Exception: {e} | field: {field} | value: {value}")

print(m_beach_message)


Station_Name: "Oak Street Weather Station"
Measurement_Timestamp {
  seconds: 1432306800
}
Wet_Bulb_Temperature: 7
Humidity: 55
Total_Rain: 1.4
Wind_Direction: 63
Wind_Speed: 1.9
Maximum_Wind_Speed: 2.8
Solar_Radiation: 780
Heading: 322
Battery_Life: 12
Measurement_Timestamp_Label: "05/22/2015 3:00 PM"
Measurement_ID: "OakStreetWeatherStation201505221500"



Сериализую и десериализую

In [11]:
with open("m_beach.bin", "wb") as f:
    print("write as binary")
    bytesAsString = m_beach_message.SerializeToString()
    f.write(bytesAsString)


with open("m_beach.bin", "rb") as f:
    print("read values")
    simple_message_read = m_beach_pb2.m_beach_message().FromString(f.read())


print(f"\nOBJECT LENGTH: {len(bytesAsString)}\nFILE SIZE: {os.path.getsize('m_beach.bin')}\n{bytesAsString}")


write as binary
read values

OBJECT LENGTH: 131
FILE SIZE: 131
b'\n\x1aOak Street Weather Station\x12\x06\x08\xf0\x88\xfd\xaa\x05%\x00\x00\xe0@(7E33\xb3?P?]33\xf3?e333@p\x8c\x06x\xc2\x02\x85\x01\x00\x00@A\x8a\x01\x1205/22/2015 3:00 PM\x92\x01#OakStreetWeatherStation201505221500'


In [12]:
class _JSONDecoder(json.JSONDecoder):
    def __init__(self, *args, **kwargs):
        json.JSONDecoder.__init__(
            self, object_hook=self.object_hook, *args, **kwargs)

    def object_hook(self, obj):
        ret = {}
        for key, value in obj.items():
            if key in {'timestamp', 'whatever'}:
                ret[key] = datetime.fromisoformat(value) 
            else:
                ret[key] = value
        return ret

    
class _JSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (datetime.date, datetime.datetime, pd.Timestamp, )):
            return obj.isoformat()
        return json.JSONEncoder.default(obj)
    

msg = df.to_dict(orient='records')[0]

with open("m_beach.json", "w+") as f:
    json.dump(msg, f, cls=_JSONEncoder)

msg = json.dumps(msg, cls=_JSONEncoder)

print(f'OBJECT LENGTH: {len(msg)}\nFILE SIZE: {os.path.getsize("m_beach.json")}\n{msg}')

OBJECT LENGTH: 522
FILE SIZE: 522
{"Station_Name": "Oak Street Weather Station", "Measurement_Timestamp": "2015-05-22T15:00:00", "Air_Temperature": null, "Wet_Bulb_Temperature": 7.0, "Humidity": 55, "Rain_Intensity": 0.0, "Interval_Rain": 0.0, "Total_Rain": 1.4, "Precipitation_Type": 0, "Wind_Direction": 63, "Wind_Speed": 1.9, "Maximum_Wind_Speed": 2.8, "Barometric_Pressure": null, "Solar_Radiation": 780, "Heading": 322, "Battery_Life": 12.0, "Measurement_Timestamp_Label": "05/22/2015 3:00 PM", "Measurement_ID": "OakStreetWeatherStation201505221500"}


In [13]:
522/131 /1.4045801526717556      

2.836956521739131

<hr>

ВСЯКОЕ

In [14]:
m_beach_message.DESCRIPTOR.__dir__()

['__new__',
 'GetOptions',
 'CopyToProto',
 'EnumValueName',
 'name',
 'full_name',
 '_concrete_class',
 'file',
 'fields',
 'fields_by_name',
 'fields_by_camelcase_name',
 'fields_by_number',
 'nested_types',
 'nested_types_by_name',
 'extensions',
 'extensions_by_name',
 'extension_ranges',
 'enum_types',
 'enum_types_by_name',
 'enum_values_by_name',
 'oneofs_by_name',
 'oneofs',
 'containing_type',
 'is_extendable',
 'has_options',
 'syntax',
 '__doc__',
 '__module__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__init__',
 '__reduce_ex__',
 '__reduce__',
 '__getstate__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [15]:
print([field.name for field in m_beach_message.DESCRIPTOR.fields])


['Station_Name', 'Measurement_Timestamp', 'Air_Temperature', 'Wet_Bulb_Temperature', 'Humidity', 'Rain_Intensity', 'Interval_Rain', 'Total_Rain', 'Precipitation_Type', 'Wind_Direction', 'Wind_Speed', 'Maximum_Wind_Speed', 'Barometric_Pressure', 'Solar_Radiation', 'Heading', 'Battery_Life', 'Measurement_Timestamp_Label', 'Measurement_ID']


In [16]:
import json
df.to_dict(orient='records')[0]

{'Station_Name': 'Oak Street Weather Station',
 'Measurement_Timestamp': Timestamp('2015-05-22 15:00:00'),
 'Air_Temperature': None,
 'Wet_Bulb_Temperature': 7.0,
 'Humidity': 55,
 'Rain_Intensity': 0.0,
 'Interval_Rain': 0.0,
 'Total_Rain': 1.4,
 'Precipitation_Type': 0,
 'Wind_Direction': 63,
 'Wind_Speed': 1.9,
 'Maximum_Wind_Speed': 2.8,
 'Barometric_Pressure': None,
 'Solar_Radiation': 780,
 'Heading': 322,
 'Battery_Life': 12.0,
 'Measurement_Timestamp_Label': '05/22/2015 3:00 PM',
 'Measurement_ID': 'OakStreetWeatherStation201505221500'}

# Compression

In [17]:
protobuf_messge = bytesAsString
pb_msg = protobuf_messge

In [61]:
import sys

# sys.path.append(os.path.abspath(".\\libs\\PyComP")) 
sys.path
# sys.path.remove(os.path.abspath(".\\libs"))

['c:\\Users\\MANDR\\Desktop\\py\\1210',
 'c:\\Users\\MANDR\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip',
 'c:\\Users\\MANDR\\AppData\\Local\\Programs\\Python\\Python312\\DLLs',
 'c:\\Users\\MANDR\\AppData\\Local\\Programs\\Python\\Python312\\Lib',
 'c:\\Users\\MANDR\\AppData\\Local\\Programs\\Python\\Python312',
 '',
 'C:\\Users\\MANDR\\AppData\\Roaming\\Python\\Python312\\site-packages',
 'c:\\Users\\MANDR\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages',
 'c:\\Users\\MANDR\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\win32',
 'c:\\Users\\MANDR\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\win32\\lib',
 'c:\\Users\\MANDR\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\Pythonwin',
 'c:\\Users\\MANDR\\Desktop\\py\\1210\\libs\\PyComP']

In [28]:
from collections import Counter

import zlib, gzip, bz2, lzma

In [ ]:
pb_ratio_zlib      = len(zlib.compress(pb_msg)) / len(pb_msg)
pb_ratio_gzip      = len(gzip.compress(pb_msg)) / len(pb_msg)
pb_ratio_bz2       = len(bz2.compress(pb_msg))  / len(pb_msg)
pb_ratio_lzma      = len(lzma.compress(pb_msg)) / len(pb_msg)

print(f'pb_ratio_zlib: {pb_ratio_zlib}\
      \npb_ratio_gzip: {pb_ratio_gzip}\
      \npb_ratio_bz2:  {pb_ratio_bz2}\
      \npb_ratio_lzma: {pb_ratio_lzma}'
)

pb_ratio_zlib: 0.9236641221374046      
pb_ratio_gzip: 1.015267175572519      
pb_ratio_bz2:  1.3587786259541985      
pb_ratio_lzma: 1.4045801526717556


In [74]:
j_msg = bytes(msg, "utf-8")

j_ratio_zlib      = len(zlib.compress(j_msg)) / len(j_msg)
j_ratio_gzip      = len(gzip.compress(j_msg)) / len(j_msg)
j_ratio_bz2       = len(bz2.compress(j_msg))  / len(j_msg)
j_ratio_lzma      = len(lzma.compress(j_msg)) / len(j_msg)

print(f'j_ratio_zlib: {j_ratio_zlib}\
      \nj_ratio_gzip: {j_ratio_gzip}\
      \nj_ratio_bz2:  {j_ratio_bz2}\
      \nj_ratio_lzma: {j_ratio_lzma}'
)

j_ratio_zlib: 0.5862068965517241      
j_ratio_gzip: 0.6091954022988506      
j_ratio_bz2:  0.657088122605364      
j_ratio_lzma: 0.7279693486590039


In [101]:
import importlib
importlib.reload(huffman)

counter = Counter(pb_msg)

h = huffman.Huffman([chr(i) for i in counter.keys()], list(counter.values()))



In [102]:
pb_msg

b'\n\x1aOak Street Weather Station\x12\x06\x08\xf0\x88\xfd\xaa\x05%\x00\x00\xe0@(7E33\xb3?P?]33\xf3?e333@p\x8c\x06x\xc2\x02\x85\x01\x00\x00@A\x8a\x01\x1205/22/2015 3:00 PM\x92\x01#OakStreetWeatherStation201505221500'

In [103]:
''.join([chr(i) for i in counter.keys()])

'\n\x1aOak StreWhion\x12\x06\x08ð\x88ýª\x05%\x00à@(7E3³?P]óp\x8cxÂ\x02\x85\x01A\x8a05/21:M\x92#'

In [104]:
h.encode(pb_msg)

msg: b'\n\x1aOak Street Weather Station\x12\x06\x08\xf0\x88\xfd\xaa\x05%\x00\x00\xe0@(7E33\xb3?P?]33\xf3?e333@p\x8c\x06x\xc2\x02\x85\x01\x00\x00@A\x8a\x01\x1205/22/2015 3:00 PM\x92\x01#OakStreetWeatherStation201505221500'
self.ht: {'a': '00000', '@': '000010', '#': '0000110', '\x92': '0000111', '5': '00010', ' ': '00011', 't': '0010', 'e': '0011', 'M': '0100000', ':': '0100001', '\x8a': '0100010', 'A': '0100011', '\x85': '0100100', '\x02': '0100101', 'Â': '0100110', 'x': '0100111', '\x8c': '0101000', 'p': '0101001', 'ó': '0101010', ']': '0101011', '³': '0101100', 'E': '0101101', '7': '0101110', '(': '0101111', 'à': '0110000', '%': '0110001', '\x05': '0110010', 'ª': '0110011', 'ý': '0110100', '\x88': '0110101', 'ð': '0110110', '\x08': '0110111', '\x1a': '0111000', '\n': '0111001', '/': '011101', 'P': '011110', '\x06': '011111', '\x12': '100000', 'n': '100001', 'o': '100010', 'i': '100011', 'h': '100100', 'W': '100101', 'k': '100110', 'O': '100111', '\x00': '10100', 'r': '10101', '0': '1

KeyError: 10

In [105]:
msg = b'\n\x1aOak Street Weather Station\x12\x06\x08\xf0\x88\xfd\xaa\x05%\x00\x00\xe0@(7E33\xb3?P?]33\xf3?e333@p\x8c\x06x\xc2\x02\x85\x01\x00\x00@A\x8a\x01\x1205/22/2015 3:00 PM\x92\x01#OakStreetWeatherStation201505221500'


In [107]:
for m in msg:
    print(m)

10
26
79
97
107
32
83
116
114
101
101
116
32
87
101
97
116
104
101
114
32
83
116
97
116
105
111
110
18
6
8
240
136
253
170
5
37
0
0
224
64
40
55
69
51
51
179
63
80
63
93
51
51
243
63
101
51
51
51
64
112
140
6
120
194
2
133
1
0
0
64
65
138
1
18
48
53
47
50
50
47
50
48
49
53
32
51
58
48
48
32
80
77
146
1
35
79
97
107
83
116
114
101
101
116
87
101
97
116
104
101
114
83
116
97
116
105
111
110
50
48
49
53
48
53
50
50
49
53
48
48
